# Exercici 1

Descarrega el dataset adjunt de dades oficials de la UEFA i selecciona un atribut del conjunt de dades. Calcula el p-valor i digues si rebutja la hipòtesi nul·la agafant un alfa de 5%. 

In [3]:
# Importem les llibreries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Eliminem limitacions en l'amplada de les columnes
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)

# Carreguem el dataset i el visualitzem
ruta = 'C:/Users/Ariadna/Desktop/IT ACADEMY/ESPECIALITZACIÓ/SPRINT 5/ENTREGA SPRINT 5/Lligues europees UEFA.csv'
df_uefa = pd.read_csv(ruta, sep=';', encoding='latin2') # Especifiquem separadors i encoding
df_uefa.head() # Mostrem els primers resultats

,Rk,Squad,Country,LgRk,MP,W,D,L,GF,GA,GD,Pts,Pts/G,xG,xGA,xGD,xGD/90,Last 5,Attendance,Top Team Scorer,Goalkeeper
0,1,Manchester City,ENG,1,37,28,6,3,96,24,72,90,2.43,86.1,26.8,59.3,1.60,W W W W D,52739,Kevin De Bruyne - 15,Ederson
1,2,Liverpool,ENG,2,36,26,8,2,89,24,65,86,2.39,84.6,33.1,51.4,1.43,W W W D W,53367,Mohamed Salah - 22,Alisson
2,3,Real Madrid,ESP,1,37,26,7,4,80,31,49,85,2.30,73.0,45.8,27.2,0.73,W W L W D,40624,Karim Benzema - 27,Thibaut Courtois
3,4,Bayern Munich,GER,1,34,24,5,5,97,37,60,77,2.26,88.1,37.1,51.0,1.50,W W L D D,33176,Robert Lewandowski - 35,Manuel Neuer
4,5,Paris S-G,FRA,1,37,25,8,4,85,36,49,83,2.24,71.6,38.1,33.4,0.90,W D D D W,41188,Kylian Mbappé - 25,Keylor Navas


In [4]:
# Observem el tipus de variables del dataset per entendre de què es tracten
df_uefa.dtypes

Rk                   int64
Squad               object
Country             object
LgRk                 int64
MP                   int64
W                    int64
D                    int64
L                    int64
GF                   int64
GA                   int64
GD                   int64
Pts                  int64
Pts/G              float64
xG                 float64
xGA                float64
xGD                float64
xGD/90             float64
Last 5              object
Attendance           int64
Top Team Scorer     object
Goalkeeper          object
dtype: object

Primerament descriurem les variables per poder seleccionar un atribut que ens interessi.

1. Rk (Classificació): Un nombre enter que representa la classificació de l'equip o el conjunt en una lliga o competició.

2. Squad (Equip): Un camp de text o objecte que representa el nom de l'equip de futbol.

3. Country (País): El país o la nació associada a l'equip.

4. LgRk (Classificació de Lliga): Un nombre enter que representa la classificació de l'equip dins de la lliga.

5. MP (Partits Jugats): El nombre de partits jugats per l'equip.

6. W (Victòries): El nombre de partits guanyats per l'equip.

7. D (Empats): El nombre de partits que han acabat en empat per l'equip.

8. L (Derrotes): El nombre de partits perduts per l'equip.

9. GF (Gols a Favor): El nombre total de gols marcats per l'equip.

10. GA (Gols en Contra): El nombre total de gols encaixats per l'equip.

11. GD (Diferència de Gols): La diferència entre els gols marcats (GF) i els gols encaixats (GA).

12. Pts (Punts): El nombre total de punts guanyats per l'equip a la lliga o competició.

13. Pts/G (Punts Per Partit): El nombre mitjà de punts guanyats per partit.

14. xG (Gols Esperats): Una mesura estadística que representa el nombre de gols esperats que un equip hauria de marcar basant-se en diversos factors.

15. xGA (Gols Esperats en Contra): El nombre de gols esperats que un equip hauria d'encaixar basant-se en diversos factors.

16. xGD (Diferència de Gols Esperada): La diferència entre els gols esperats marcats (xG) i els gols esperats encaixats (xGA).

17. xGD/90 (Diferència de Gols Esperada per 90 Minuts): La diferència de gols esperada per cada 90 minuts de joc.

18. Last 5 (Últims 5): Variable categòrica que ens dóna informació sobre el rendiment de l'equip en els darrers 5 partits (si ha guanyat o ha perdut). 

19. Attendance (Assistència): El nombre d'espectadors o seguidors que han assistit als partits de l'equip.

20. Top Team Scorer (Golejador Principal): El nom del màxim golejador de l'equip.

21. Goalkeeper (Porter): El nom del porter de l'equip.

Com hem pogut veure, el dataset conté informació sobre la UEFA, incloent dades sobre el rendiment de l'equip, estadístiques i altres detalls com l'assistència del públic. Ens pot servir doncs, per analitzar el rendiment dels diferents equips. 

En aquest cas, hem decidit analitzar si l'assistència de públic influeix en el rendiment de l'equip. Per tant, seleccionarem la variable independent 'Attendance' (Assistència) i la variable dependent 'Pts' (Punts). Amb l'objectiu d'obtenir aquesta informació concreta, definim el test d'hipòtesi següent:
- Ho (Hipòtesi nul.la): No hi ha relació entre les variables seleccionades. És a dir, el nivell d'assistència de públic no influeix en el rendiment de l'equip.
- Ha (Hipòtesi alternativa): Hi ha una relació entre les variables seleccionades. És a dir, el nivell d'assistència influeix en el rendiment de l'equip.

Ara farem un anàlisi de regressió per provar la relació entre l'assistència i els punts guanyats per l'equip, que ens proporcionarà el coeficient per a l'assistència, juntament amb el seu p-valor. D'aquesta manera podrem avaluar la significació estadística de la relació entre aquestes variables.

In [5]:
# Importem la llibreria necessària per fer els càlculs
import statsmodels.api as sm

# Defineim les variables independents i dependents
X = df_uefa['Attendance']  # Variable independent (assistència)
y = df_uefa['Pts']         # Variable dependent (punts)

# Afegim una constant per al model
X = sm.add_constant(X)

# Creem el model de regressió
model = sm.OLS(y, X).fit()

# Obtenim els resultats de la regressió
resultats = model.summary()

# Mostrem els resultats
print(resultats)


                            OLS Regression Results                            
Dep. Variable:                    Pts   R-squared:                       0.336
Model:                            OLS   Adj. R-squared:                  0.329
Method:                 Least Squares   F-statistic:                     48.48
Date:                Tue, 17 Oct 2023   Prob (F-statistic):           4.14e-10
Time:                        16:35:03   Log-Likelihood:                -391.25
No. Observations:                  98   AIC:                             786.5
Df Residuals:                      96   BIC:                             791.7
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         34.4736      2.568     13.424      0.0

Els resultats de l'anàlisi de regressió són els següents:

R-squared (R²): Aquesta és la proporció de la variabilitat de la variable dependent (Pts) que s'explica per la variable independent (Attendance). En aquest cas, R² és 0,336, el que significa que aproximadament el 33,6% de la variabilitat en els punts de l'equip es pot explicar per l'assistència. Això suggereix que hi ha una correlació moderada entre l'assistència i els punts de l'equip.

F-statistic: Aquest valor estadístic és utilitzat per avaluar la significació global del model de regressió. En aquest cas és 48,48, i el p-valor associat és 4.14e-10 (pràcticament zero). Això indica que el model de regressió és estadísticament significatiu i que l'ús de l'assistència com a variable independent té un impacte significatiu en la predicció dels punts de l'equip.

Coeficients:
   - El coeficient constant (const) és 34,4736. Això significa que quan l'assistència és zero, es pot esperar que l'equip tingui al voltant de 34,47 punts.
   - El coeficient de l'assistència (Attendance) és 0,0006. Això indica que per cada unitat addicional d'assistència, es pot esperar un augment de 0,0006 punts en la puntuació de l'equip.

P-value: El p-valor associat al coeficient de l'assistència és 0,000, el que és molt menor que el nivell de significació típic d'α = 0,05. Segons els resultats de l'anàlisi de regressió, l'assistència té un impacte significatiu en els punts de l'equip. El model de regressió és estadísticament significatiu, i l'assistència té una influència positiva en el rendiment de l'equip en terme de punts.

Finalment, volem saber si amb un nivell de significació del 5% (α = 0.05), el p-valor rebutja la hipòtesi nul.la. En aquest cas, el p-valor és 0.000, que és menor que α = 0.05. Això significa que podem rebutjar la hipòtesi nul.la i que per tant, hi ha prou evidència estadística per concloure que l'assistència influeix en el rendiment de l'equip en terme de punts amb un nivell de confiança del 95%.

# Exercici 2

Amb el mateix dataset selecciona dos altres atributs del conjunt de dades. Calcula els p-valors i digues si rebutgen la hipòtesi nul·la agafant un alfa de 5%.

Ara analitzarem si la diferència de gols (GD) està relacionada amb el golejador principal (Top Team Scorer) en terme de punts obtinguts per l'equip.

In [6]:
# En aquest cas calcularem el coeficient de Pearson i el seu p-valor associat
# Importem la llibreria
from scipy.stats import pearsonr

# Seleccionem les variables GD i Pts i les guardem en una variable
dades = df_uefa[['GD', 'Pts']]

# Calculem el coeficient de correlació de Pearson i el p-valor
correlacio, p_value = pearsonr(dades['GD'], dades['Pts'])

# Mostrem el resultat
print(f"Coeficient de correlació de Pearson entre GD i Pts: {correlacio:.2f}")
print(f"P-valor: {p_value:.5f}")


Coeficient de correlació de Pearson entre GD i Pts: 0.96
P-valor: 0.00000


Com a segona variable a analitzar juntament amb la diferència de gols (GD) i els punts (Pts), analitzarem "xG", que representa el nombre de gols esperats que un equip hauria de marcar. Això ens pot ajudar a comprendre si l'equip aprofita al màxim les seves oportunitats o si està jugant per sota de les expectatives. Amb tot, analitzarem com la diferència entre el rendiment real i el rendiment esperat està relacionada amb els punts de l'equip.

In [7]:
# Seleccionem les variables xG i Pts i les guardem en una variable
dades_2 = df_uefa[['xG', 'Pts']]

# Calculem el coeficient de correlació de Pearson i el p-valor
correlacio, p_value = pearsonr(dades_2['xG'], dades_2['Pts'])

# Mostrem el resultat
print(f"Coeficient de correlació de Pearson entre xG i Pts: {correlacio:.2f}")
print(f"P-valor: {p_value:.5f}")

Coeficient de correlació de Pearson entre xG i Pts: 0.85
P-valor: 0.00000


Ara valorarem i compararem els resultats obtinguts. Per una banda, el coeficient de correlació de Pearson entre 'GD' i 'Pts' és 0.96. Això indica una correlació altament positiva i significativa entre aquestes dues variables. En altres paraules, a mesura que la diferència de gols augmenta (sigui a favor de l'equip), també augmenten els punts obtinguts per l'equip.
El p-valor és 0.00000, el que significa que la correlació és estadísticament significativa. No hi ha evidència per rebutjar la hipòtesi que hi ha una forta relació entre la diferència de gols i els punts.

Per una altra banda, el coeficient de correlació de Pearson entre xG i Pts és 0.85. Això indica també una correlació positiva i significativa entre aquestes dues variables, és a dir que les xifres de gols esperats estan relacionades amb el nombre de punts obtinguts per l'equip. El p-valor també és 0.00000, la qual cosa suggereix que la correlació és estadísticament significativa (no hi ha evidència per rebutjar la hipòtesi que hi ha una relació entre els gols esperats i els punts.

En resum, ambdues variables estan fortament correlacionades amb els punts de l'equip i les correlacions són estadísticament significatives. 

# Exercici 3

Continua amb el conjunt de dades adjunt i selecciona tres atributs del conjunt de dades. Calcula el p-valor i digues si rebutja la hipòtesi nul·la agafant un alfa de 5%.



Els tres atributs que seleccionarem seran:
- xGA (expected goals against, és a dir els gols que s'espera rebre de l'equip contrari)
- Attendance (assistència)
- Last 5 (últims 5 partits) 

Pel que fa a la variable Last 5, la convertirem en puntuació numèrica (ara la tenim com a W/D/L: win/draw/lose). La codificació serà 3 punts si l'equip guanya, 1 punt si l'equip empata i 0 punts si l'equip perd.

De cara a l'anàlisi, per a cada atribut, realitzarem una regressió lineal amb Pts (Punts) com a variable dependent i calcularem el p-valor associat amb el coeficient de la variable independent.

In [11]:
# Importem la llibreria
import statsmodels.api as sm 

# Fem l'anàlisi per a la variable xGA
X1 = df_uefa['xGA']  # Seleccionem la columna 'xGA' com a variable independent
X1 = sm.add_constant(X1)  # Afegim una constant per tenir un intercepte en la regressió
y = df_uefa['Pts']  # Seleccionem la columna 'Pts' com a variable dependent
model1 = sm.OLS(y, X1).fit()  # Realitzem la regressió lineal
p_value_xGA = model1.pvalues['xGA']  # Obtenim el p-valor associat amb la variable xGA

# Fem l'anàlisi per a la variable Attendance
X2 = df_uefa['Attendance']  # Seleccionem la columna 'Attendance' com a variable independent
X2 = sm.add_constant(X2)  # Afegim una constant per tenir un intercepte en la regressió
model2 = sm.OLS(y, X2).fit()  # Realitzem la regressió lineal
p_value_Attendance = model2.pvalues['Attendance']  # Obtenim el p-valor associat amb la variable Attendance

# Fem l'anàlisi per a la variable Last 5 (codificada)
df_uefa['Last5_score'] = df_uefa['Last 5'].apply(lambda x: x.count('W')*3 + x.count('D'))  # Codifiquem resultats
X3 = df_uefa['Last5_score']  # Seleccionem la columna 'Last5_score' com a variable independent
X3 = sm.add_constant(X3)  # Afegim una constant per tenir un intercepte en la regressió
model3 = sm.OLS(y, X3).fit()  # Realitzem la regressió lineal
p_value_Last5 = model3.pvalues['Last5_score']  # Obtenim el p-valor associat amb la variable Last5_score

# Mostrem els resultats
print(f"P-valor per xGA: {p_value_xGA}")
print(f"P-valor per Attendance: {p_value_Attendance}")
print(f"P-valor per Last 5: {p_value_Last5}")

P-valor per xGA: 3.5498199799103233e-22
P-valor per Attendance: 4.138186008962836e-10
P-valor per Last 5: 1.0253373817491734e-14


 Basant-nos en els p-valors que hem obtingut (valors molt baixos), podem concloure que les tres variables (xGA, Attendance i Last 5) estan significativament relacionades amb els punts obtinguts. Aquestes relacions són estadísticament significatives amb un nivell de confiança molt alt. En els tres casos rebutgem la hipòtesi nul.la perquè el p-valor és inferior al nivell de significació del 5%.